# Carga de Datos


## .xls / HTML a .csv
Los datos del Mineduc se exportan como archivos .xls, realmente teniendo contenido de HTML. Debido a eso, debemos de parsear el archivo HTML e identificar la tabla correcta que contiene los datos. Luego, podemos exportar los datos a su archivo .csv correspondiente, utilizando el valor de la columna 'Departamento' para nombrarlo. Adicionalmente, los archivos utilizan un encoding diferente al estándar utf-8, por lo cual vamos a especificarlo al momento de leer los archivos HTML.

In [31]:
import pandas as pd
from pathlib import Path
from collections import defaultdict
import re


def parse_html_excel_file(file_path):
    file_path = Path(file_path)

    try:
        tables = pd.read_html(str(file_path), encoding="iso-8859-1")

        if not tables:
            return {"success": False, "error": "No tables found in HTML"}

        required_headers = ["CODIGO", "DISTRITO", "DEPARTAMENTO", "MUNICIPIO"]
        target_table = None
        target_index = None

        for i, df in enumerate(tables):
            df_columns_upper = [str(col).upper().strip() for col in df.columns]
            if all(header in df_columns_upper for header in required_headers):
                target_table = df
                target_index = i
                break
            else:
                if len(df) > 0:
                    first_row_upper = [
                        str(cell).upper().strip() for cell in df.iloc[0]
                    ]
                    if all(header in first_row_upper for header in required_headers):
                        df.columns = df.iloc[0]
                        df = df.drop(df.index[0]).reset_index(drop=True)
                        target_table = df
                        target_index = i
                        break

        if target_table is None:
            return {
                "success": False,
                "error": "No table found with required headers.",
            }

        target_table = target_table.dropna(how="all")

        return {
            "success": True,
            "data": target_table,
            "table_index": target_index,
            "total_tables": len(tables),
        }

    except Exception as e:
        return {"success": False, "error": str(e)}


def sanitize_filename(text):
    filename = text.lower().replace(" ", "_")
    return re.sub(r"[^\w_.]", "", filename)


def process_html_files_directory(input_dir, output_dir):
    input_path = Path(input_dir)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)

    files = list(input_path.glob("*.xls"))

    print(f"Found {len(files)} .xls files to process")

    successful_files = []
    failed_files = []
    all_departamentos = defaultdict(list)

    for file_path in files:
        result = parse_html_excel_file(file_path)

        if result["success"]:
            df = result["data"]

            departamentos = []
            for departamento, group in df.groupby("DEPARTAMENTO"):
                filename = f"datos_{sanitize_filename(departamento)}.csv"

                output_file = output_path / filename
                group.to_csv(output_file, index=False)

                departamentos.append(
                    {"name": departamento, "filename": filename, "rows": len(group)}
                )

                all_departamentos[departamento].append(
                    {
                        "source_file": file_path.name,
                        "csv_file": filename,
                        "rows": len(group),
                    }
                )

            successful_files.append(
                {
                    "file": file_path.name,
                    "departamentos": departamentos,
                    "total_rows": len(df),
                }
            )

            print(f"  ✅ Processed {file_path.name}: Created {len(departamentos)} CSVs.")

        else:
            failed_files.append({"file": file_path.name, "error": result["error"]})
            print(f"  ❌ Failed {file_path.name}: {result['error']}")

    print("\n" + "=" * 60)
    print("PROCESSING SUMMARY")
    print("=" * 60)

    print(f"\nTotal files: {len(files)}")
    print(f"✅ Successful: {len(successful_files)}")
    print(f"❌ Failed: {len(failed_files)}")

    # The detailed successful/failed files lists will only be printed if there are successful/failed files.
    # The summary already gives a count, so the repetition for successful files is removed here.
    # The detailed list for failed files remains as it provides useful error messages.
    if failed_files:
        print(f"\n❌ FAILED FILES:")
        for item in failed_files:
            print(f"  - {item['file']}: {item['error']}")

    duplicates = {
        name: sources
        for name, sources in all_departamentos.items()
        if len(sources) > 1
    }
    if duplicates:
        print(f"\n⚠️ DUPLICATE DEPARTAMENTOS:")
        for dept_name, sources in duplicates.items():
            print(f"  {dept_name}: appears in {len(sources)} files")
    else:
        print("\n✅ No duplicate departamentos found")


test_result = parse_html_excel_file("data/raw/establecimiento.xls")
if test_result["success"]:
    print("\nSingle file test successful!")

    print("\n" + "=" * 60)
    print("PROCESSING ALL FILES")
    print("=" * 60)
    process_html_files_directory("data/raw", "data/csv")
else:
    print(f"❌ Single file test failed: {test_result['error']}")


Single file test successful!

PROCESSING ALL FILES
Found 23 .xls files to process
  ✅ Processed establecimiento (12).xls: Created 1 CSVs.
  ✅ Processed establecimiento (18).xls: Created 1 CSVs.
  ✅ Processed establecimiento (14).xls: Created 1 CSVs.
  ✅ Processed establecimiento (10).xls: Created 1 CSVs.
  ✅ Processed establecimiento (5).xls: Created 1 CSVs.
  ✅ Processed establecimiento.xls: Created 1 CSVs.
  ✅ Processed establecimiento (11).xls: Created 1 CSVs.
  ✅ Processed establecimiento (22).xls: Created 1 CSVs.
  ✅ Processed establecimiento (9).xls: Created 1 CSVs.
  ✅ Processed establecimiento (3).xls: Created 1 CSVs.
  ✅ Processed establecimiento (13).xls: Created 1 CSVs.
  ✅ Processed establecimiento (21).xls: Created 1 CSVs.
  ✅ Processed establecimiento (17).xls: Created 1 CSVs.
  ✅ Processed establecimiento (20).xls: Created 1 CSVs.
  ✅ Processed establecimiento (19).xls: Created 1 CSVs.
  ✅ Processed establecimiento (16).xls: Created 1 CSVs.
  ✅ Processed establecimiento

## .csv a DataFrames
Luego de haber creado los archivos, podemos cargarlos a DataFrames para realizar el análisis necesario.

In [32]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
from collections import Counter, defaultdict
import matplotlib.pyplot as plt

# Load all CSV files
csv_dir = Path("data/csv")
csv_files = list(csv_dir.glob("*.csv"))

print(f"Found {len(csv_files)} CSV files")

# Load all datasets
datasets = {}
for csv_file in csv_files:
    dataset_name = csv_file.stem
    datasets[dataset_name] = pd.read_csv(csv_file)

print(f"Loaded {len(datasets)} datasets")

Found 23 CSV files
Loaded 23 datasets


# Descripción

## Filas y Columnas

In [33]:
shape_info = []
for name, df in datasets.items():
    shape_info.append({
        'dataset': name,
        'rows': df.shape[0],
        'columns': df.shape[1]
    })

shape_df = pd.DataFrame(shape_info)
print("📊 Dataset Shapes:")
print(shape_df.sort_values('rows', ascending=False))

print(f"\n📈 Summary:")
print(f"Total rows across all datasets: {shape_df['rows'].sum():,}")
print(f"Average rows per dataset: {shape_df['rows'].mean():.0f}")
print(f"Min/Max rows: {shape_df['rows'].min()} / {shape_df['rows'].max()}")

📊 Dataset Shapes:
                 dataset  rows  columns
8        datos_guatemala  1036       17
11  datos_ciudad_capital   864       17
3       datos_san_marcos   431       17
7        datos_escuintla   393       17
5   datos_quetzaltenango   365       17
14   datos_chimaltenango   300       17
20   datos_suchitepequez   296       17
21         datos_jutiapa   296       17
15   datos_huehuetenango   295       17
2     datos_alta_verapaz   294       17
13          datos_izabal   273       17
17      datos_retalhuleu   272       17
18           datos_peten   270       17
1     datos_sacatepequez   208       17
16          datos_quiche   184       17
0       datos_chiquimula   136       17
6       datos_santa_rosa   133       17
12          datos_jalapa   121       17
22          datos_solola   111       17
9      datos_el_progreso    97       17
19    datos_baja_verapaz    94       17
4           datos_zacapa    70       17
10     datos_totonicapan    51       17

📈 Summary:
Total rows

## Integridad de los Datos

### Consistencia en Nombres de Columnas

In [34]:
all_columns = []
column_consistency = {}

for name, df in datasets.items():
    columns = list(df.columns)
    all_columns.append(columns)
    column_consistency[name] = columns

first_columns = all_columns[0]
all_same = all(columns == first_columns for columns in all_columns)

if all_same:
    print(f"\n✅ Standard columns ({len(first_columns)}):")
    for i, col in enumerate(first_columns, 1):
        print(f"  {i:2d}. {col}")
else:
    print("\n❌ Column differences found:")
    for name, columns in column_consistency.items():
        if columns != first_columns:
            print(f"  {name}: {columns}")


✅ Standard columns (17):
   1. CODIGO
   2. DISTRITO
   3. DEPARTAMENTO
   4. MUNICIPIO
   5. ESTABLECIMIENTO
   6. DIRECCION
   7. TELEFONO
   8. SUPERVISOR
   9. DIRECTOR
  10. NIVEL
  11. SECTOR
  12. AREA
  13. STATUS
  14. MODALIDAD
  15. JORNADA
  16. PLAN
  17. DEPARTAMENTAL


### Encoding Problemático
Como mencionamos anteriormente, el encoding de los archivos originales era distinto de "utf-8". Nos dimos cuenta al realizar el análisis sobre el encoding problemático, sin embargo al cambiarlo dentro de la función anterior logramos correr con éxito este análisis.

In [35]:
import pandas as pd
from collections import defaultdict
import re

problematic_char = "�"
all_problematic_samples = defaultdict(list)
issue_found = False

for dataset_name, df in datasets.items():
    for col in df.columns:
        if df[col].dtype == "object":
            str_series = df[col].astype(str)

            contains_char_mask = str_series.str.contains(problematic_char, na=False)

            if contains_char_mask.any():
                issue_found = True
                current_samples = str_series[contains_char_mask].unique().tolist()
                for sample_val in current_samples:
                    if len(all_problematic_samples[col]) < 5:
                        all_problematic_samples[col].append(sample_val)


if issue_found:
    print(f"❌ Encoding issues found (char: '{problematic_char}'):")
    sorted_cols_with_issues = sorted(all_problematic_samples.keys())

    for col in sorted_cols_with_issues:
        samples = all_problematic_samples[col]
        print(f"  Column '{col}':")
        for val in samples:
            print(f"    • {val}")
else:
    print(f"✅ No encoding issues found (char: '{problematic_char}')")

✅ No encoding issues found (char: '�')


# Análisis de Variables
Las variables que más operaciones de limpieza necesitan son:

In [36]:
summary_stats = []

for col in first_columns:
    col_data = []
    for name, df in datasets.items():
        if col in df.columns:
            series = df[col].astype(str).str.strip()
            col_data.extend(series)

    series_all = pd.Series(col_data)
    n_total = len(series_all)
    n_missing = (series_all == "").sum() + series_all.isna().sum()
    n_unique = series_all.nunique()

    summary_stats.append({
        "column": col,
        "missing (%)": round((n_missing / n_total) * 100, 2),
        "unique_values": n_unique,
        "sample_values": series_all.dropna().unique()[:5].tolist()
    })

df_summary = pd.DataFrame(summary_stats)
df_summary.sort_values("missing (%)", ascending=False, inplace=True)

display(df_summary)

,column,missing (%),unique_values,sample_values
0,CODIGO,0.0,6590,"[20-01-0024-46, 20-01-0029-46, 20-01-0030-46, ..."
1,DISTRITO,0.0,620,"[20-001, 20-024, 20-027, 99-001, 20-030]"
2,DEPARTAMENTO,0.0,23,"[CHIQUIMULA, SACATEPEQUEZ, ALTA VERAPAZ, SAN M..."
3,MUNICIPIO,0.0,343,"[CHIQUIMULA, SAN JOSE LA ARADA, SAN JUAN ERMIT..."
4,ESTABLECIMIENTO,0.0,3779,"[ESCUELA DE CIENCIAS COMERCIALES NOCTURNA, INS..."
5,DIRECCION,0.0,4428,"[10A. AVENIDA 3-71 ZONA 1, 2A CALLE ENTRE 11 Y..."
6,TELEFONO,0.0,4208,"[79422150.0, 79420395.0, 41942927.0, 79420290...."
7,SUPERVISOR,0.0,598,"[CESAR ADALBERTO NOGUERA JACOME, SILVIA MARILE..."
8,DIRECTOR,0.0,3860,"[HÉCTOR ALIDIO CERON BRENES, ROMEO RIVERA CHAC..."
9,NIVEL,0.0,1,[DIVERSIFICADO]


## Código
Este valor parece ser un identificador único, queremos explorar las siguientes propiedades:
- Unicidad: Este código es único dentro de su respectivo dataset o todos?
- Formato: Es consistente el formato en todos los datasets? Existen errores de digitación?
- Valores Faltantes: Existen valores faltantes?
- Nos ayuda a identificar únicamente a los demás valores?

Debido a esto, queremos realizar los siguientes pasos de limpieza:

- Revisar errores de digitación, cómo lo pueden ser whitespaces o formato incongruente
- Identificar Valores Faltantes
- Identificar unicidad del código
- Identificar si cada código corresponde a un set de valores único de las demás columnas

## Distrito
Este valor parece ser un identificador geográfico, siguiendo un formato XX-YYY. Queremos explorar las siguientes propiedades
- Formato: Es consistente el formato?
- Valores Faltantes: Existen valores faltantes dentro de los datasets?
- Nos ayuda a identificar únicamente algún otro valor?

Debido a esto, queremos realizar los siguientes pasos de limpieza:

- Identificar valores faltantes
- Revisar errores de digitación
- Enforzar un formato consistente
- Identificar si nos puede ayuda a verificar la consistencia de Municipio o algún otro valor

## Departamento

Ya que los DataFrames se encuentran divididos por departamento, esta entrada debería ser completamente consistente. Además, podemos proponer los siguientes pasos para una mayor consistencia:

- Identificar el valor RAW más común, ya que un error de digitación sería menos frecuente
- Tomar ese valor y realizar las siguientes transformaciones
    - Conversión a minúsculas
    - Reemplazo de espacios por _
    - Aplicar a todas las columnas de cada DF individual
- Aplicar OHE luego de mergear los DFs

## Municipio

Este valor representa la división política a nivel municipal. Dado que se usa como categoría geográfica clave, es esencial asegurar consistencia.

Exploración a realizar:

- Formato: ¿Se encuentra todo en mayúsculas? ¿Hay tildes inconsistentes?
- Valores Faltantes: Confirmar si hay celdas vacías o marcadas incorrectamente.
- Relación con otros campos: ¿El municipio concuerda con el departamento correspondiente?

Pasos de limpieza propuestos:

- Convertir todo el texto a mayúsculas (.str.upper()).
- Eliminar espacios extra antes o después (.str.strip()).
- Normalizar tildes y caracteres especiales si necesario.
- Validar los municipios contra una lista oficial por departamento.


## Establecimiento

Nombre propio de la institución educativa. Puede contener muchas variantes tipográficas y estilísticas que dificultan análisis posteriores.

Exploración a realizar:

- Formato: ¿Se mantiene una capitalización consistente?
- Valores Faltantes: ¿Hay registros sin nombre?
- Redundancia o duplicación: ¿Existen nombres duplicados con leves diferencias?

Pasos de limpieza propuestos:

- Capitalizar nombres con .str.title() para uniformidad visual.
- Eliminar espacios repetidos entre palabras.
- Remover puntuación innecesaria.
- Establecer reglas para abreviaciones comunes si se encuentran (ej. "Inst." por "Instituto").

## DIRECCIÓN

Campo libre con alta variabilidad. Las direcciones pueden contener múltiples abreviaturas, puntuación, y errores de digitación.

Exploración a realizar:
- Formato: ¿Existen patrones comunes? ¿Se usan abreviaturas (CALLE, AV, etc.)?
- Valores inconsistentes: ¿Uso indistinto de mayúsculas, puntuación, acentos?

Pasos de limpieza propuestos:
- Normalizar a mayúsculas (.str.upper()).
- Crear reglas de sustitución para abreviaturas frecuentes (ej. AVENIDA → AV.).
- Quitar símbolos innecesarios y estandarizar signos de puntuación.
- Opcional: usar expresiones regulares para separar partes de la dirección (calle, número, zona, etc.).

## TELÉFONO

Campo numérico con alta variabilidad en formato. Puede contener números concatenados, separadores o caracteres no numéricos.

Exploración a realizar:
- Formato: ¿Todos los valores contienen exactamente 8 dígitos? ¿Hay múltiples números por celda?
- Errores: ¿Caracteres no numéricos? ¿Espacios o signos innecesarios?

Pasos de limpieza propuestos:

- Remover todos los caracteres no numéricos con re.sub(r"\D", "", telefono).
- Validar longitud estándar (8 dígitos en Guatemala).
- Si hay múltiples números, dividir y elegir el primero o guardarlos como lista.

## SUPERVISOR

Nombre del supervisor responsable. Puede presentar variaciones por uso de tildes, mayúsculas, y errores ortográficos menores.

Exploración a realizar:

- Formato: ¿Mayúsculas/minúsculas inconsistentes?
- Duplicación: ¿Un mismo nombre aparece escrito de múltiples formas?

Pasos de limpieza propuestos:

- Capitalizar con .str.title() para uniformidad.
- Remover espacios dobles y caracteres extraños.
- Normalizar tildes si es necesario.

## DIRECTOR

Similar al campo de supervisor, representa nombres propios con riesgos similares de inconsistencia.

Exploración a realizar:

- Aplicar los mismos criterios que SUPERVISOR.

Pasos de limpieza propuestos:

- Igual estrategia de capitalización, limpieza de espacios, y normalización de caracteres.

## NIVEL
Este campo tiene un único valor: "DIVERSIFICADO".

Exploración a realizar:

- Verificar que efectivamente todos los valores son iguales.
- Confirmar si es útil conservar esta columna.

Pasos de limpieza propuestos:

- Si es redundante, considerar eliminarla para evitar ruido en análisis futuros.

## SECTOR

Categoría con pocos valores únicos. Se espera valores como "OFICIAL", "PRIVADO", etc.

Exploración a realizar:

- ¿Mayúsculas o minúsculas inconsistentes?
- ¿Errores ortográficos?

Pasos de limpieza propuestos:

- Convertir todo a mayúsculas y eliminar espacios (.str.upper().str.strip()).
- Validar los valores contra un conjunto permitido: {OFICIAL, PRIVADO, MUNICIPAL, COOPERATIVA}.
- Convertir a tipo Categorical.

## ÁREA
Identifica si la institución está en zona rural o urbana.

Exploración a realizar:

- Confirmar que los valores son: URBANA, RURAL, SIN ESPECIFICAR.
- Verificar errores de digitación o combinaciones no válidas.

Pasos de limpieza propuestos:

- Uniformar mayúsculas (.str.upper()).
- Reemplazar variantes de “sin especificar” por un valor estándar (ej. "DESCONOCIDO").
- Convertir a Categorical.

## STATUS
Campo con un único valor: "ABIERTA".

Exploración a realizar:

Verificar si realmente todos los valores son iguales.

Evaluar su utilidad en análisis futuros.

Pasos de limpieza propuestos:

Eliminar si es redundante (sin variabilidad).

## MODALIDAD
Pocos valores únicos: "MONOLINGUE", "BILINGUE".

Exploración a realizar:

Verificar mayúsculas y tildes.

Validar que no existan variantes escritas incorrectamente.

Pasos de limpieza propuestos:

Uniformar mayúsculas y acentos (.str.upper()).

Reemplazar variantes con un mapeo fijo.

Convertir a Categorical

## JORNADA
Categoría horaria. Existen valores como “MATUTINA”, “VESPERTINA”, “DOBLE”, “NOCHE”, etc.

Exploración a realizar:

Verificar consistencia de términos.

Identificar redundancias o términos similares con diferencias menores.

Pasos de limpieza propuestos:

Normalizar formato (.str.upper().str.strip()).

Mapear variantes a un conjunto estándar.

Eliminar signos extra o abreviaciones inconsistentes.

## PLAN
Puede incluir valores con paréntesis, como “DIARIO(REGULAR)”, que dificultan análisis.

Exploración a realizar:

¿Hay signos innecesarios como paréntesis o guiones?

¿Hay términos redundantes?

Pasos de limpieza propuestos:

Eliminar paréntesis y su contenido con str.replace(r'\(.*?\)', '').

Eliminar espacios extra y convertir a mayúsculas.

Validar valores contra una lista limpia predefinida.

Convertir a Categorical.

## DEPARTAMENTAL
Representa una división regional administrativa.

Exploración a realizar:

Verificar mayúsculas, errores tipográficos.

Confirmar que corresponde con el valor del campo DEPARTAMENTO.

Pasos de limpieza propuestos:

Uniformar texto (.str.upper().str.strip()).

Validar contra un listado oficial del MINEDUC.

Opcional: cruzar con DEPARTAMENTO para consistencia.

## Definición de funciones para limpieza de datos

# Operaciones de Limpieza

## Código

Como primer paso, buscamos identificar errores de digitación. Para esto, vamos a hacer lo siguiente:

- Eliminar los whitespaces dentro de todas las columnas
- Definir un regex que matchee el formato 16-02-0020-46 WW-XX-YYYY-ZZ, para asegurarnos que los datos hayan sido ingresados correctamente.

In [37]:
import re

pattern = r'^\d{2}-\d{2}-\d{4}-\d{2}$'

total_rows = 0
total_valid = 0
total_invalid = 0

for name, df in datasets.items():
    if 'CODIGO' not in df.columns:
        print(f"[{name}] No 'CODIGO' column found.")
        continue

    # Clean whitespace and replace original CODIGO with cleaned version
    df['CODIGO_clean'] = df['CODIGO'].astype(str).str.replace(r'\s+', '', regex=True)
    df['CODIGO'] = df['CODIGO_clean']

    # Check format on cleaned CODIGO
    df['is_valid_format'] = df['CODIGO'].apply(lambda x: bool(re.match(pattern, x)))

    total = len(df)
    valid = df['is_valid_format'].sum()
    invalid = total - valid

    total_rows += total
    total_valid += valid
    total_invalid += invalid

    # Drop helper columns, keep cleaned CODIGO only
    df.drop(columns=['CODIGO_clean', 'is_valid_format'], inplace=True)

print(f"\n=== TOTAL across all datasets ===")
print(f"Total rows: {total_rows} | Valid CODIGO: {total_valid} | Invalid CODIGO: {total_invalid}")



=== TOTAL across all datasets ===
Total rows: 6590 | Valid CODIGO: 6590 | Invalid CODIGO: 0


Luego de haber verificado que cada código sigue el formato correcto, podemos verificar la unicidad del código a lo largo de los diferentes datasets.

In [38]:
import pandas as pd

# Storage for all CODIGO values
all_codigos = []

# Track problems
duplicates_within = {}
missing_codigo = {}

# Analyze each dataset
for name, df in datasets.items():
    if 'CODIGO' not in df.columns:
        print(f"[WARNING] Dataset {name} is missing 'CODIGO' column.")
        continue

    # Clean and standardize CODIGO
    df['CODIGO'] = df['CODIGO'].astype(str).str.strip()

    # Store all codes
    all_codigos.extend(df['CODIGO'].tolist())

    # Check for duplicates within dataset
    duplicated = df['CODIGO'][df['CODIGO'].duplicated()]
    if not duplicated.empty:
        duplicates_within[name] = duplicated.value_counts()

    # Check for missing
    missing = df['CODIGO'].isna().sum() + (df['CODIGO'] == '').sum()
    if missing > 0:
        missing_codigo[name] = missing

# Create a full Series of all codes
codigo_series = pd.Series(all_codigos)

# Check for duplicates across datasets
global_dupes = codigo_series[codigo_series.duplicated(keep=False)]

# Ensure non_unique_codigos_across is always a Series
non_unique_codigos_across = (
    pd.Series(global_dupes.value_counts()) if not global_dupes.empty
    else pd.Series([], dtype=int)
)

# Summary report
print("\n=== Summary ===")
print(f"Total codes across all datasets: {len(codigo_series)}")
print(f"Unique codes: {codigo_series.nunique()}")
print(f"Duplicated across datasets: {len(non_unique_codigos_across)}")

if duplicates_within:
    print("\n❗ Duplicates *within* datasets:")
    for name, series in duplicates_within.items():
        print(f"  - {name}: {series.sum()} duplicates")

if missing_codigo:
    print("\n❗ Missing CODIGO values:")
    for name, count in missing_codigo.items():
        print(f"  - {name}: {count} missing")

if not non_unique_codigos_across.empty:
    print("\n❗ Duplicated CODIGO *across* datasets (top 10):")
    print(non_unique_codigos_across.head(10))



=== Summary ===
Total codes across all datasets: 6590
Unique codes: 6590
Duplicated across datasets: 0


Por último, podemos verificar que cada código corresponda a una y solo una combinación de todas las demás variables.

In [39]:
from collections import defaultdict
import pandas as pd

# Step 1: Combine all datasets into a single DataFrame
all_data = []
for name, df in datasets.items():
    df = df.copy()
    df["source"] = name  # to track origin
    all_data.append(df)
all_df = pd.concat(all_data, ignore_index=True)

# Step 2: Drop CODIGO and group by the remaining columns
grouped = all_df.groupby(all_df.columns.difference(["CODIGO", "source"]).tolist())

# Step 3: Collect entries where multiple CODIGOs share the same other data
duplicates_except_codigo = defaultdict(list)
for _, group in grouped:
    if group["CODIGO"].nunique() > 1:
        duplicates_except_codigo[len(duplicates_except_codigo)] = group

# Step 4: Show sample results
print(f"Found {len(duplicates_except_codigo)} groups with duplicated rows apart from CODIGO.\n")

# Display first few examples
for i, df in list(duplicates_except_codigo.items())[:5]:
    print(f"Group {i+1}:")
    print(df[["CODIGO"] + [col for col in df.columns if col != "CODIGO"]].to_string(index=False))
    print("-" * 60)


Found 134 groups with duplicated rows apart from CODIGO.

Group 1:
       CODIGO DISTRITO DEPARTAMENTO MUNICIPIO                       ESTABLECIMIENTO                         DIRECCION TELEFONO                           SUPERVISOR                           DIRECTOR         NIVEL  SECTOR  AREA  STATUS  MODALIDAD    JORNADA            PLAN DEPARTAMENTAL          source
05-07-0063-46   05-012    ESCUINTLA LA GOMERA COLEGIO MIXTO "LUIS CARDOZA Y ARAGÓN" 4A AVENIDA 5-112 BARRIO CHIPILAPA 78423518 BRENDA YOHANNA GUDIEL LOPEZ DE MUÑOZ KIMBERLY YAMILETH ESPINOZA ESCOBAR DIVERSIFICADO PRIVADO RURAL ABIERTA MONOLINGUE VESPERTINA DIARIO(REGULAR)     ESCUINTLA datos_escuintla
05-07-0066-46   05-012    ESCUINTLA LA GOMERA COLEGIO MIXTO "LUIS CARDOZA Y ARAGÓN" 4A AVENIDA 5-112 BARRIO CHIPILAPA 78423518 BRENDA YOHANNA GUDIEL LOPEZ DE MUÑOZ KIMBERLY YAMILETH ESPINOZA ESCOBAR DIVERSIFICADO PRIVADO RURAL ABIERTA MONOLINGUE VESPERTINA DIARIO(REGULAR)     ESCUINTLA datos_escuintla
------------------------

Esto nos indica que existen entradas duplicadas con códigos diferentes. De momento, este output no nos es realmente útil. Al no tener estandarizadas las demás columnas, puede que todavía haya establecimientos escritos de manera diferente. Sin embargo, esto nos ayuda a determinar que la variable "código" no nos es realmente útil para identificar errores de digitación en otros campos.

## Distrito

## Departamento

El tener datasets separados por departamento nos ayuda un poco con la limpieza, podemos simplemente eliminar trailing / leading whitespaces, convertir a letras mayúsculas e identificar que los valores sean únicos dentro de cada dataset.

In [40]:
departamentos = set()

for name, df in datasets.items():
    if 'DEPARTAMENTO' not in df.columns:
        print(f"[{name}] No 'DEPARTAMENTO' column found.")
        continue

    # Clean in-place: strip whitespace and uppercase
    df['DEPARTAMENTO'] = df['DEPARTAMENTO'].astype(str).str.strip().str.upper()

    # Collect unique cleaned departamentos for checking
    unique_depts = df['DEPARTAMENTO'].unique()
    departamentos.update(unique_depts)

print("Unique DEPARTAMENTO values across all datasets:")
for dept in sorted(departamentos):
    print(f"- {dept}")

Unique DEPARTAMENTO values across all datasets:
- ALTA VERAPAZ
- BAJA VERAPAZ
- CHIMALTENANGO
- CHIQUIMULA
- CIUDAD CAPITAL
- EL PROGRESO
- ESCUINTLA
- GUATEMALA
- HUEHUETENANGO
- IZABAL
- JALAPA
- JUTIAPA
- PETEN
- QUETZALTENANGO
- QUICHE
- RETALHULEU
- SACATEPEQUEZ
- SAN MARCOS
- SANTA ROSA
- SOLOLA
- SUCHITEPEQUEZ
- TOTONICAPAN
- ZACAPA


Verificando los resultados, la limpieza fue exitosa y se comparte un formato consistente e único dentro de cada dataset.

## Municipio

In [41]:
import unicodedata
from difflib import get_close_matches

def normalize_string(s):
    s = str(s).strip().upper()
    s = unicodedata.normalize('NFD', s)
    s = ''.join(c for c in s if unicodedata.category(c) != 'Mn')  # Remove accents
    return s

for name, df in datasets.items():
    if 'MUNICIPIO' not in df.columns or 'DEPARTAMENTO' not in df.columns:
        print(f"[{name}] Missing 'MUNICIPIO' or 'DEPARTAMENTO' column.")
        continue

    # Normalize MUNICIPIO column only
    df['MUNICIPIO'] = df['MUNICIPIO'].apply(normalize_string)

    print(f"\n[{name}] Similar MUNICIPIO names within each DEPARTAMENTO:")

    for dept in df['DEPARTAMENTO'].dropna().unique():
        df_dept = df[df['DEPARTAMENTO'] == dept]
        municipios = df_dept['MUNICIPIO'].dropna().unique()

        checked = set()
        for mun in municipios:
            if mun in checked:
                continue
            matches = get_close_matches(mun, municipios, n=5, cutoff=0.85)
            matches = [m for m in matches if m != mun and m not in checked]
            if matches:
                print(f"- {dept}: {mun} ~ {', '.join(matches)}")
                checked.update(matches)
            checked.add(mun)



[datos_chiquimula] Similar MUNICIPIO names within each DEPARTAMENTO:

[datos_sacatepequez] Similar MUNICIPIO names within each DEPARTAMENTO:

[datos_alta_verapaz] Similar MUNICIPIO names within each DEPARTAMENTO:

[datos_san_marcos] Similar MUNICIPIO names within each DEPARTAMENTO:

[datos_zacapa] Similar MUNICIPIO names within each DEPARTAMENTO:

[datos_quetzaltenango] Similar MUNICIPIO names within each DEPARTAMENTO:

[datos_santa_rosa] Similar MUNICIPIO names within each DEPARTAMENTO:

[datos_escuintla] Similar MUNICIPIO names within each DEPARTAMENTO:

[datos_guatemala] Similar MUNICIPIO names within each DEPARTAMENTO:

[datos_el_progreso] Similar MUNICIPIO names within each DEPARTAMENTO:

[datos_totonicapan] Similar MUNICIPIO names within each DEPARTAMENTO:

[datos_ciudad_capital] Similar MUNICIPIO names within each DEPARTAMENTO:
- CIUDAD CAPITAL: ZONA 1 ~ ZONA 21, ZONA 19, ZONA 18, ZONA 17
- CIUDAD CAPITAL: ZONA 2 ~ ZONA 24, ZONA 12
- CIUDAD CAPITAL: ZONA 3 ~ ZONA 13
- CIUDAD CA

Al ver los resultados, podemos ver que los únicos posibles errores de digitación son las similitudes entre "Zona 1" y "Zona 19" por ejemplo.

## Establecimiento

Primero, podemos empezar "limpiando" el texto al remover caracteres especiales que puede no se hayan incluido en todas las entradas para un mismo establecimiento. Además, vamos a eliminar espacios adicionales por medio trim y trabajar únicamente con mayúsculas.

In [42]:
import unicodedata
import string
from difflib import get_close_matches
import pandas as pd

def clean_text(text):
    """Cleans text in place by:
    1. Removing accents/diacritics
    2. Standardizing punctuation and spaces
    3. Converting to uppercase"""
    if pd.isna(text):
        return text
    text = unicodedata.normalize('NFKD', str(text)).encode('ASCII', 'ignore').decode()
    text = text.translate(str.maketrans('', '', string.punctuation + '"“”‘’'))
    text = ' '.join(text.split())
    return text.upper()

# Clean and analyze in one pass
for name, df in datasets.items():
    if 'MUNICIPIO' not in df.columns or 'ESTABLECIMIENTO' not in df.columns:
        print(f"[{name}] Missing required columns.")
        continue

    # DIRECTLY MODIFY ORIGINAL COLUMNS
    df['MUNICIPIO'] = df['MUNICIPIO'].apply(clean_text)
    df['ESTABLECIMIENTO'] = df['ESTABLECIMIENTO'].apply(clean_text)

    print(f"\n[{name}] Similar ESTABLECIMIENTO names within each MUNICIPIO:")

    # Use the cleaned original columns
    municipios = df['MUNICIPIO'].dropna().unique()
    for mun in municipios:
        df_mun = df[df['MUNICIPIO'] == mun]
        establecimientos = df_mun['ESTABLECIMIENTO'].dropna().unique()

        checked = set()
        matches_found = []

        for est in establecimientos:
            if est in checked:
                continue
            matches = get_close_matches(est, establecimientos, n=5, cutoff=0.95)
            matches = [m for m in matches if m != est and m not in checked]
            if matches:
                matches_found.append((est, matches))
                checked.update(matches)
            checked.add(est)

        if matches_found:
            print(f"\nMunicipio: {mun}")
            for est, matched in matches_found:
                print(f"  Base: {est}")
                print(f"  Similar: {', '.join(matched)}")


[datos_chiquimula] Similar ESTABLECIMIENTO names within each MUNICIPIO:

[datos_sacatepequez] Similar ESTABLECIMIENTO names within each MUNICIPIO:

[datos_alta_verapaz] Similar ESTABLECIMIENTO names within each MUNICIPIO:

Municipio: COBAN
  Base: LICEO PREUNIVERSITARIO DE NORTE
  Similar: LICEO PREUNIVERSITARIO DEL NORTE

Municipio: SANTA MARIA CAHABON
  Base: CENTRO DE ESTUDIOS PREUNIVERSITARIOS CAHABON CEPREC
  Similar: CENTRO DE ESTUDIOS PREUNIVERSITARIO CAHABON CEPREC

[datos_san_marcos] Similar ESTABLECIMIENTO names within each MUNICIPIO:

Municipio: COMITANCILLO
  Base: COLEGIO INTERCIENTIFICO MULTILINGUE Y MULTIVERSIDAD MUNDO REAL
  Similar: COLEGIO INTERCIENTIFICOMULTILINGUE Y MULTIDIVERSIDAD MUNDO REAL

Municipio: EL TUMBADOR
  Base: COLEGIO PRIVADO MIXTO IMPERIAL
  Similar: COLEGIO PRIVADO MIXTO IMPERIAL2

Municipio: SAN JOSE OJETENAM
  Base: INSTITUTO DE EDUCACION DIVERSIFICADA POR EL SISTEMA DE COOPERATIVA DE SENANZA CESAR PAULA MAES
  Similar: INSTITUTO DE EDUCACION DIVE

Podemos ver que todavía tenemos múltiples establecimientos con nombres similares, esto lo vamos a dejar así de momento y buscamos utilizar otras variables para identificar un mismo establecimiento escrito de múltiples maneras. En algunos casos, logramos identificar errores pequeños como "NO 3" y "NO3" que indica nombres diferentes. En estos casos, buscamos utilizar otras variables identificadores cómo teléfono y dirección para matchear nombres iguales.

## Dirección

Este campo es bastante complicado, por lo que vamos a tomar un approach iterativo. Es decir, primero vamos a empezar con estandarización básica y luego vamos a revisar direcciones similares dentro de un mismo municipio. Esto nos permitirá ver errores que no habíamos visto anteriormente. Primero, vamos a reemplazar abreviaciones comunes como

- AV - Avenida
- COL - Colonia
- DIAG - DIAGNOAL

In [43]:
import pandas as pd
import re

def clean_address(address):
    if pd.isna(address): return ""
    addr = str(address).upper().strip()

    # Protect kilometer markers
    km_markers = re.findall(r'KM\s*\d+\.?\d*', addr)
    for km in km_markers:
        addr = addr.replace(km, km.replace(' ', '_'))

    # Standard replacements
    replacements = [
        (r',', ''),  # Remove all commas
        (r'\bAV\.?\b', 'AVENIDA'),
        (r'\bC\.?\b', 'CALLE'),
        (r'\bCL\.?\b', 'CALLE'),
        (r'\bDIAG\.?\b', 'DIAGONAL'),
        (r'\bCOL\.?\b', 'COLONIA'),
        (r'(\d+)[A-Za-z]*\.?', r'\1'),
        (r'\.(?!\d)', ''),  # Remove periods not before numbers
        (r'\s+', ' ')
    ]

    for pat, repl in replacements:
        addr = re.sub(pat, repl, addr)

    # Restore KM markers
    for km in km_markers:
        clean_km = km.replace('_', ' ').replace('. ', '.').replace(' ', '')
        addr = addr.replace(km.replace(' ', '_'), clean_km)

    addr = re.sub(r'Z\.?\b', 'ZONA', addr)
    addr = re.sub(r'(\d+)\s*-\s*(\d+)', r'\1-\2', addr)

    return addr

# Apply cleaning DIRECTLY to DIRECCION column
for name, df in datasets.items():
    if 'DIRECCION' in df.columns:
        df['DIRECCION'] = df['DIRECCION'].apply(clean_address)

# Verification: Print original and modified addresses from first 3 datasets
print("=== ADDRESS CLEANING VERIFICATION ===")
for name, df in list(datasets.items())[:3]:
    if 'DIRECCION' in df.columns:
        samples = df[['DIRECCION']].dropna().head(2)
        print(f"\nDataset: {name}")
        for addr in samples['DIRECCION']:
            print(f"  {addr}")

=== ADDRESS CLEANING VERIFICATION ===

Dataset: datos_chiquimula
  10 AVENIDA 3-71 ZONA 1
  2 CALLE ENTRE 11 Y 12 AVENIDAS ZONA 1

Dataset: datos_sacatepequez
  5 CALLE ORIENTE NO.17
  3 CALLE ORIENTE NO.11

Dataset: datos_alta_verapaz
  KM.2 SALIDA A SAN JUAN CHAMELCO ZONA 8
  KM_2095 ENTRADA A LA CIUDAD


Luego de limpiar las direcciones, podemos utilizar una función para intentar identificar direcciones similares

In [44]:
def find_similar_addresses(datasets, cutoff=0.9):
    repeated = 0
    for name, df in datasets.items():
        if 'MUNICIPIO' not in df.columns or 'DIRECCION' not in df.columns:
            continue

        print(f"\n[{name}] Potential address typos:")

        municipios = df['MUNICIPIO'].dropna().unique()
        for mun in municipios:
            df_mun = df[df['MUNICIPIO'] == mun]
            addresses = df_mun['DIRECCION'].dropna().unique()

            checked = set()
            matches_found = []

            for addr in addresses:
                if addr in checked:
                    continue

                # Standardize connectors for better matching
                standardized = re.sub(r'\b(Y|CON)\b', '', addr)

                # Filter candidates (same street type and zone)
                street_type = re.search(r'(AVENIDA|CALLE|DIAGONAL|COLONIA|KM)', addr)
                zone = re.search(r'ZONA \d+', addr)

                if not street_type or not zone:
                    continue

                candidates = [
                    a for a in addresses
                    if street_type.group() in a
                    and zone.group() in a
                    and a != addr  # Exclude self
                ]

                # Compare standardized versions
                matches = [
                    m for m in candidates
                    if get_close_matches(standardized, [re.sub(r'\b(Y|CON)\b', '', m)], n=1, cutoff=cutoff)
                ]

                if matches:
                    matches_found.append((addr, matches))
                    checked.update(matches)
                    repeated += 1

                checked.add(addr)

            if matches_found:
                print(f"\nMunicipio: {mun}")
                for addr, matched in matches_found:
                    print(f"  {addr}")
                    print(f"  → Possible variants: {', '.join(matched)}\n")
    print(repeated)

# Usage:
find_similar_addresses(datasets)


[datos_chiquimula] Potential address typos:

Municipio: CHIQUIMULA
  4 AVENIDA 4-20 ZONA 1
  → Possible variants: 8 AVENIDA 4-40 ZONA 1, 6 AVENIDA 4-10 ZONA 1

  1 CALLE 5-61 ZONA 2
  → Possible variants: 1 CALLE 5-62 ZONA 2

  3 AVENIDA 4-49 ZONA 1
  → Possible variants: 8 AVENIDA 4-40 ZONA 1

  8 AVENIDA 6-60 ZONA 1
  → Possible variants: 8 AVENIDA 4-40 ZONA 1


Municipio: ESQUIPULAS
  6 AVENIDA 6-99 ZONA 1 BARRIO SAN JOAQUIN
  → Possible variants: 6 AVENIDA 6-99 ZONA 1 BARRIO SAN JOAQUÍN


Municipio: QUEZALTEPEQUE
  9 CALLEJON 1-72 ZONA 1
  → Possible variants: 9 CALLEJÓN 1-72 ZONA 1


Municipio: IPALA
  1 DIAGONAL 2-44 ZONA 1
  → Possible variants: 1 DIAGONAL 2-32 ZONA 1


[datos_sacatepequez] Potential address typos:

Municipio: SAN LUCAS SACATEPEQUEZ
  1 AVENIDA Y 6 CALLE 1-80 ZONA 1
  → Possible variants: 1 AVENIDA 6 CALLE 1-80 ZONA 1


Municipio: CIUDAD VIEJA
  3 CALLE 1-22 ZONA 3
  → Possible variants: 3 CALLE1-22 ZONA 3


[datos_alta_verapaz] Potential address typos:

Munici

Basado en el output, nos podemos dar cuenta que hay algunas modificaciones que podemos realizar todavía:

- Palabras / letras conectivas, por ejemplo "1 AVENIDA Y 6 CALLE 1-80 ZONA 1" y "1 AVENIDA 6 CALLE 1-80 ZONA 1"
- Caracteres especiales para los ordinales, cómo "8ª CALLE ZONA 1 BARRIO LATINO"

Las demás modificaciones presentan bastantes complicaciones al intentar generalizarlas, por lo que terminaremos de limpiar las demás columnas e intentaremos identificar establecimientos con la misma dirección para intervenir "manualmente" y establecer una única dirección.


In [45]:
import re
import unicodedata
import pandas as pd

def refine_address(address):
    if not address or pd.isna(address):
        return address

    # Convert to string and normalize special characters
    addr = str(address)

    # Remove accents and diacritics (like clean_text does)
    addr = unicodedata.normalize('NFKD', addr).encode('ASCII', 'ignore').decode()

    # Handle ordinal indicators (ª, º)
    addr = re.sub(r'(\d+)(ª|º)\b', r'\1', addr)

    # Standardize connectors (Y, CON) - replace with space
    addr = re.sub(r'\b(Y|CON)\b', ' ', addr)

    # Remove other special characters (preserving hyphens between numbers)
    # First protect number-hyphen-number patterns
    protected = re.findall(r'\d+-\d+', addr)
    for i, ph in enumerate(protected):
        addr = addr.replace(ph, f'__PROTECTED_{i}__')

    # Remove punctuation except protected patterns
    addr = re.sub(r'[^\w\s-]', ' ', addr)

    # Restore protected patterns
    for i, ph in enumerate(protected):
        addr = addr.replace(f'__PROTECTED_{i}__', ph)

    # Clean up spaces
    addr = re.sub(r'\s+', ' ', addr).strip().upper()

    return addr

# Apply the additional cleaning to all datasets
for name, df in datasets.items():
    if 'DIRECCION' in df.columns:
        df['DIRECCION'] = df['DIRECCION'].apply(refine_address)

# Verification: Print modified addresses from first 3 datasets
print("\n=== REFINED ADDRESS CLEANING VERIFICATION ===")
for name, df in list(datasets.items())[:3]:
    if 'DIRECCION' in df.columns:
        samples = df[['DIRECCION']].dropna().head(3)
        print(f"\nDataset: {name}")
        for addr in samples['DIRECCION']:
            print(f"  {addr}")


=== REFINED ADDRESS CLEANING VERIFICATION ===

Dataset: datos_chiquimula
  10 AVENIDA 3-71 ZONA 1
  2 CALLE ENTRE 11 12 AVENIDAS ZONA 1
  2 CALLE 12 AVENIDA ZONA 1

Dataset: datos_sacatepequez
  5 CALLE ORIENTE NO 17
  3 CALLE ORIENTE NO 11
  4 AVENIDA NORTE N-26

Dataset: datos_alta_verapaz
  KM 2 SALIDA A SAN JUAN CHAMELCO ZONA 8
  KM_2095 ENTRADA A LA CIUDAD
  7 AVENIDA 11-109 ZONA 6


## Telefono

In [46]:
import pandas as pd
import re

def validate_phones(df):
    if 'TELEFONO' not in df.columns:
        return "No phone column"

    valid_count = 0
    invalid_count = 0
    problem_samples = []

    for phone in df['TELEFONO']:
        # Handle NaN/empty first
        if pd.isna(phone):
            invalid_count += 1
            continue

        # Convert to string, strip whitespace
        phone_str = str(phone).strip()

        # Remove .0 if caused by float
        if phone_str.endswith('.0'):
            phone_str = phone_str[:-2]

        # Remove ALL non-digit characters
        digits_only = re.sub(r'[^\d]', '', phone_str)

        # Validate
        if len(digits_only) == 8:
            valid_count += 1
        else:
            invalid_count += 1
            if len(problem_samples) < 3:
                problem_samples.append(phone_str)

    result = f"✅ {valid_count} valid | ❌ {invalid_count} invalid"
    if problem_samples:
        result += f"\n   Sample problems: {problem_samples}"
    return result

# === FULL LOOP TO PRINT RESULTS FOR EACH DF ===
print("=== TRUE PHONE VALIDATION ===")
for name, df in datasets.items():
    if 'TELEFONO' in df.columns:
        print(f"{name.ljust(25)} {validate_phones(df)}")


=== TRUE PHONE VALIDATION ===
datos_chiquimula          ✅ 129 valid | ❌ 7 invalid
datos_sacatepequez        ✅ 207 valid | ❌ 1 invalid
   Sample problems: ['52660000-78323083']
datos_alta_verapaz        ✅ 286 valid | ❌ 8 invalid
   Sample problems: ['79504027-79504028', '4085613']
datos_san_marcos          ✅ 427 valid | ❌ 4 invalid
   Sample problems: ['574479']
datos_zacapa              ✅ 70 valid | ❌ 0 invalid
datos_quetzaltenango      ✅ 365 valid | ❌ 0 invalid
datos_santa_rosa          ✅ 132 valid | ❌ 1 invalid
datos_escuintla           ✅ 390 valid | ❌ 3 invalid
   Sample problems: ['78880429-78880419']
datos_guatemala           ✅ 1024 valid | ❌ 12 invalid
   Sample problems: ['2437011', '24342036-24123888', '24314989-24310955']
datos_el_progreso         ✅ 93 valid | ❌ 4 invalid
   Sample problems: ['4215928', '3033']
datos_totonicapan         ✅ 46 valid | ❌ 5 invalid
   Sample problems: ['56769964-77663283', '56769964-7766328', '77663283-77663283']
datos_ciudad_capital      ✅ 860 va

Nos podemos dar cuenta de 2 cosas, el principal "error" son los establecimientos con dos números telefónicos. Lo que podemos hacer, es tomar 2 números telefónicos como válidos y si un establecimiento cuenta con 2 números telefónicos mal ingresados (ej. 4210394058993785) separarlos utilizando un -.

In [47]:
import re
import pandas as pd

import re
import pandas as pd

def fix_phones(df):
    if 'TELEFONO' not in df.columns:
        return df

    for i, phone in df['TELEFONO'].items():
        if pd.isna(phone):
            continue

        phone_str = str(phone).strip()

        if phone_str.endswith('.0'):
            phone_str = phone_str[:-2]

        digit_chunks = re.findall(r'\d{5,}', phone_str)

        final_numbers = []

        if len(digit_chunks) == 1 and len(digit_chunks[0]) >= 14:
            candidates = re.findall(r'\d{8}', digit_chunks[0])
            if len(candidates) >= 2:
                final_numbers = candidates[:2]

        else:
            # Solo tomar los números exactos de 8 dígitos
            final_numbers = [n for n in digit_chunks if len(n) == 8][:2]

        if final_numbers:
            df.at[i, 'TELEFONO'] = '-'.join(final_numbers)
        else:
            # En lugar de borrar, dejamos el valor original intacto para que validación lo detecte
            df.at[i, 'TELEFONO'] = phone_str

    return df


# Primero lo arreglás
for name, df in datasets.items():
    if 'TELEFONO' in df.columns:
        fix_phones(df)

/tmp/ipykernel_120124/1746684611.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '79422150' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, 'TELEFONO'] = '-'.join(final_numbers)
/tmp/ipykernel_120124/1746684611.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '77601385' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, 'TELEFONO'] = '-'.join(final_numbers)
/tmp/ipykernel_120124/1746684611.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '79414031' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, 'TELEFONO'] = '-'.join(final_numbers)
/tmp/ipykernel_120124/1746684611.py:34: FutureWarni

Ahora podemos utilizar una función de validación actualizada, dónde se toman como válidos varios números telefónicos y podemos asegurar que los pendientes son teléfonos faltantes o mal

In [48]:
import pandas as pd
import re

def validate_phones(df):
    if 'TELEFONO' not in df.columns:
        return "No phone column"

    valid_count = 0
    invalid_count = 0
    problem_samples = []

    for phone in df['TELEFONO']:
        if pd.isna(phone) or not str(phone).strip():
            invalid_count += 1
            continue

        phone_str = str(phone).strip()

        # Remove .0 if float artifact
        if phone_str.endswith('.0'):
            phone_str = phone_str[:-2]

        # Split multiple possible phones
        parts = re.split(r'[-/\s]', phone_str)

        part_has_valid = False
        for part in parts:
            digits_only = re.sub(r'\D', '', part)
            if len(digits_only) == 8:
                valid_count += 1
                part_has_valid = True
            elif digits_only:
                invalid_count += 1
                # Add example only if there's at least something wrong but present
                if len(problem_samples) < 3 and phone_str not in problem_samples:
                    problem_samples.append(phone_str)

        if not parts and not part_has_valid:
            invalid_count += 1

    result = f"✅ {valid_count} valid | ❌ {invalid_count} invalid"
    if problem_samples:
        result += f"\n   Sample problems: {problem_samples}"
    return result

print("=== TRUE PHONE VALIDATION ===")
for name, df in datasets.items():
    if 'TELEFONO' in df.columns:
        print(f"{name.ljust(25)} {validate_phones(df)}")


=== TRUE PHONE VALIDATION ===
datos_chiquimula          ✅ 129 valid | ❌ 7 invalid
datos_sacatepequez        ✅ 209 valid | ❌ 0 invalid
datos_alta_verapaz        ✅ 288 valid | ❌ 7 invalid
   Sample problems: ['4085613']
datos_san_marcos          ✅ 427 valid | ❌ 4 invalid
   Sample problems: ['574479']
datos_zacapa              ✅ 70 valid | ❌ 0 invalid
datos_quetzaltenango      ✅ 365 valid | ❌ 0 invalid
datos_santa_rosa          ✅ 132 valid | ❌ 1 invalid
datos_escuintla           ✅ 392 valid | ❌ 2 invalid
datos_guatemala           ✅ 1032 valid | ❌ 8 invalid
   Sample problems: ['2437011', '5510735', '5364736']
datos_el_progreso         ✅ 93 valid | ❌ 4 invalid
   Sample problems: ['4215928', '3033']
datos_totonicapan         ✅ 55 valid | ❌ 0 invalid
datos_ciudad_capital      ✅ 860 valid | ❌ 4 invalid
   Sample problems: ['2232068', '2223228', '2232379']
datos_jalapa              ✅ 122 valid | ❌ 0 invalid
datos_izabal              ✅ 285 valid | ❌ 1 invalid
   Sample problems: ['5388633']
d

In [49]:
import pandas as pd
import re

def extract_valid_phones(phone_str):
    """Extrae todos los teléfonos válidos (8 dígitos) de un string."""
    if not phone_str or pd.isna(phone_str):
        return []
    phone_str = str(phone_str).strip()
    if phone_str.endswith('.0'):
        phone_str = phone_str[:-2]
    parts = re.split(r'[-/\s]', phone_str)
    valid_phones = []
    for part in parts:
        digits_only = re.sub(r'\D', '', part)
        if len(digits_only) == 8:
            valid_phones.append(digits_only)
    return valid_phones

def build_valid_phone_reference(datasets):
    """Crea un diccionario {ESTABLECIMIENTO: lista de teléfonos válidos únicos} de todos los datasets."""
    phone_ref = {}
    for df in datasets.values():
        if all(col in df.columns for col in ['ESTABLECIMIENTO', 'TELEFONO']):
            for _, row in df.iterrows():
                name = row['ESTABLECIMIENTO']
                if pd.isna(name):
                    continue
                phones = extract_valid_phones(row['TELEFONO'])
                if not phones:
                    continue
                if name not in phone_ref:
                    phone_ref[name] = set()
                phone_ref[name].update(phones)
    # Convertir sets a listas para consistencia
    for k in phone_ref:
        phone_ref[k] = list(phone_ref[k])
    return phone_ref

def fix_invalid_phones_by_reference(datasets):
    print("=== FIXING INVALID PHONES BY ESTABLECIMIENTO REFERENCE ===")
    phone_ref = build_valid_phone_reference(datasets)
    fixed_count = 0

    for name, df in datasets.items():
        if all(col in df.columns for col in ['ESTABLECIMIENTO', 'TELEFONO']):
            for i, row in df.iterrows():
                phone_val = row['TELEFONO']
                est_name = row['ESTABLECIMIENTO']
                if pd.isna(phone_val) or not str(phone_val).strip():
                    # teléfono vacío o nulo, intentar reemplazar
                    if est_name in phone_ref:
                        new_phone = '-'.join(phone_ref[est_name])
                        df.at[i, 'TELEFONO'] = new_phone
                        fixed_count += 1
                else:
                    # teléfono presente, validar
                    valid_phones = extract_valid_phones(phone_val)
                    # Si no tiene ningún teléfono válido, intentar reemplazar
                    if len(valid_phones) == 0 and est_name in phone_ref:
                        new_phone = '-'.join(phone_ref[est_name])
                        df.at[i, 'TELEFONO'] = new_phone
                        fixed_count += 1

    print(f"Total teléfonos corregidos: {fixed_count}")
    return datasets

# Uso:
datasets = fix_invalid_phones_by_reference(datasets)

# Luego validás con tu función habitual
print("=== TRUE PHONE VALIDATION ===")
for name, df in datasets.items():
    if 'TELEFONO' in df.columns:
        print(f"{name.ljust(25)} {validate_phones(df)}")


=== FIXING INVALID PHONES BY ESTABLECIMIENTO REFERENCE ===
Total teléfonos corregidos: 47
=== TRUE PHONE VALIDATION ===
datos_chiquimula          ✅ 1213 valid | ❌ 3 invalid
datos_sacatepequez        ✅ 209 valid | ❌ 0 invalid
datos_alta_verapaz        ✅ 1016 valid | ❌ 1 invalid
datos_san_marcos          ✅ 1149 valid | ❌ 2 invalid
   Sample problems: ['574479']
datos_zacapa              ✅ 70 valid | ❌ 0 invalid
datos_quetzaltenango      ✅ 365 valid | ❌ 0 invalid
datos_santa_rosa          ✅ 133 valid | ❌ 0 invalid
datos_escuintla           ✅ 392 valid | ❌ 2 invalid
datos_guatemala           ✅ 1402 valid | ❌ 1 invalid
   Sample problems: ['40']
datos_el_progreso         ✅ 474 valid | ❌ 1 invalid
   Sample problems: ['3033']
datos_totonicapan         ✅ 55 valid | ❌ 0 invalid
datos_ciudad_capital      ✅ 866 valid | ❌ 0 invalid
datos_jalapa              ✅ 122 valid | ❌ 0 invalid
datos_izabal              ✅ 286 valid | ❌ 0 invalid
datos_chimaltenango       ✅ 308 valid | ❌ 0 invalid
datos_huehu

In [50]:
import pandas as pd
import re

def validate_phones(df):
    if 'TELEFONO' not in df.columns:
        return "No phone column"

    valid_count = 0
    invalid_count = 0
    problem_samples = []

    for phone in df['TELEFONO']:
        if pd.isna(phone) or not str(phone).strip():
            invalid_count += 1
            if len(problem_samples) < 3:
                problem_samples.append('')
            continue

        phone_str = str(phone).strip()

        # Remove .0 if float artifact
        if phone_str.endswith('.0'):
            phone_str = phone_str[:-2]

        # Split if there are multiple numbers
        parts = re.split(r'[-/\s]', phone_str)

        part_has_valid = False
        for part in parts:
            digits_only = re.sub(r'\D', '', part)
            if len(digits_only) == 8:
                valid_count += 1
                part_has_valid = True
            elif digits_only:
                invalid_count += 1
                if len(problem_samples) < 3 and phone_str not in problem_samples:
                    problem_samples.append(phone_str)

        if not part_has_valid and not parts:
            # fallback case: string with no valid digits
            invalid_count += 1
            if len(problem_samples) < 3 and phone_str not in problem_samples:
                problem_samples.append(phone_str)

    result = f"✅ {valid_count} valid | ❌ {invalid_count} invalid"
    if problem_samples:
        result += f"\n   Sample problems: {problem_samples}"
    return result

print("=== TRUE PHONE VALIDATION ===")
for name, df in datasets.items():
    if 'TELEFONO' in df.columns:
        print(f"{name.ljust(25)} {validate_phones(df)}")


=== TRUE PHONE VALIDATION ===
datos_chiquimula          ✅ 1213 valid | ❌ 3 invalid
   Sample problems: ['', '', '']
datos_sacatepequez        ✅ 209 valid | ❌ 0 invalid
datos_alta_verapaz        ✅ 1016 valid | ❌ 1 invalid
   Sample problems: ['']
datos_san_marcos          ✅ 1149 valid | ❌ 2 invalid
   Sample problems: ['', '574479']
datos_zacapa              ✅ 70 valid | ❌ 0 invalid
datos_quetzaltenango      ✅ 365 valid | ❌ 0 invalid
datos_santa_rosa          ✅ 133 valid | ❌ 0 invalid
datos_escuintla           ✅ 392 valid | ❌ 2 invalid
   Sample problems: ['', '']
datos_guatemala           ✅ 1402 valid | ❌ 1 invalid
   Sample problems: ['40']
datos_el_progreso         ✅ 474 valid | ❌ 1 invalid
   Sample problems: ['3033']
datos_totonicapan         ✅ 55 valid | ❌ 0 invalid
datos_ciudad_capital      ✅ 866 valid | ❌ 0 invalid
datos_jalapa              ✅ 122 valid | ❌ 0 invalid
datos_izabal              ✅ 286 valid | ❌ 0 invalid
datos_chimaltenango       ✅ 308 valid | ❌ 0 invalid
datos_hueh

## Supervisor

Para esta variable, realmente lo único que podemos hacer es limpiarla utilizando estandarización como letras mayúsculas. Si intentamos utilizar algunas de las estrategias anteriores, como fuzzy matching al nombre, realmente no tenemos manera de identificar que "Juan Prez" es un error de digitación para "Juan Perez".

In [51]:
import unicodedata
import string

def clean_supervisor_name(name):
    """Cleans a supervisor name by:
    1. Converting to uppercase
    2. Removing accents/diacritics
    3. Stripping whitespace
    4. Removing punctuation"""
    if pd.isna(name):
        return None

    # Convert to string, uppercase, and strip whitespace
    name = str(name).upper().strip()

    # Remove accents/diacritics
    name = unicodedata.normalize('NFKD', name)\
           .encode('ASCII', 'ignore')\
           .decode('ASCII')

    # Remove punctuation (keeps only letters and spaces)
    name = name.translate(str.maketrans('', '', string.punctuation))

    # Collapse multiple spaces
    name = ' '.join(name.split())

    return name

# Apply cleaning to SUPERVISOR column in all datasets
for df in datasets.values():
    if 'SUPERVISOR' in df.columns:
        df['SUPERVISOR'] = df['SUPERVISOR'].apply(clean_supervisor_name)

# Verification print
print("SUPERVISOR cleaning completed for all datasets.")
print("Sample cleaned names:")
for name, df in list(datasets.items())[:2]:  # Show first 2 datasets
    if 'SUPERVISOR' in df.columns:
        print(f"\n{name}:")
        print(df['SUPERVISOR'].dropna().unique()[:5])  # Show first 5 unique names

SUPERVISOR cleaning completed for all datasets.
Sample cleaned names:

datos_chiquimula:
['CESAR ADALBERTO NOGUERA JACOME' 'SILVIA MARILENA BUEZO MARTINEZ'
 'SILVIA EUGENIA VALDEZ MONTERROSO DE CARBALLO'
 'VILMA ESPERANZA LOPEZ INTERIANO' 'BYRON HORACIO SARCENO ZEPEDA']

datos_sacatepequez:
['JUAN DEMETRIO SICAJOL PEREZ' 'ZOILA ESTHELA JONFE OROZCO'
 'AURA MARINA LORENZO SAGCHE' 'OSCAR OSWALDO SON CAMEZ'
 'JUVENTINO PEREZ SICAN']


## Director

De manera similar a la anterior,  realmente lo único que podemos hacer es limpiarla utilizando estandarización como letras mayúsculas. Si intentamos utilizar algunas de las estrategias anteriores, como fuzzy matching al nombre, realmente no tenemos manera de identificar que "Juan Prez" es un error de digitación para "Juan Perez".

In [52]:
import unicodedata
import string

def clean_director_name(name):
    """Cleans a director name by:
    1. Converting to uppercase
    2. Removing accents/diacritics
    3. Stripping whitespace
    4. Removing punctuation"""
    if pd.isna(name):
        return None

    # Convert to string, uppercase, and strip whitespace
    name = str(name).upper().strip()

    # Remove accents/diacritics
    name = unicodedata.normalize('NFKD', name)\
           .encode('ASCII', 'ignore')\
           .decode('ASCII')

    # Remove punctuation (keeps only letters and spaces)
    name = name.translate(str.maketrans('', '', string.punctuation))

    # Collapse multiple spaces
    name = ' '.join(name.split())

    return name

# Apply cleaning to DIRECTOR column in all datasets
for df in datasets.values():
    if 'DIRECTOR' in df.columns:
        df['DIRECTOR'] = df['DIRECTOR'].apply(clean_director_name)

# Verification print
print("DIRECTOR cleaning completed for all datasets.")
print("Sample cleaned names:")
for name, df in list(datasets.items())[:2]:  # Show first 2 datasets
    if 'DIRECTOR' in df.columns:
        print(f"\n{name}:")
        print(df['DIRECTOR'].dropna().unique()[:5])  # Show first 5 unique names


DIRECTOR cleaning completed for all datasets.
Sample cleaned names:

datos_chiquimula:
['HECTOR ALIDIO CERON BRENES' 'ROMEO RIVERA CHACON'
 'JOSE VICENTE REYES SOLIS' 'ERWIN LEONEL COLINDRES MONROY'
 'ROXANA LISBETH URRUTIA PINTO']

datos_sacatepequez:
['JULIO ABDIEL HERNANDEZ GARCIA' 'JOSUE NATHAN SOLORZANO BERDUO'
 'MARCOS MANUEL HERNANDEZ PEREZ' 'SONIA JUDITH GALVEZ GARCIA'
 'MANUEL ANTONIO ESTRADA ORTIZ']


## Consistencia Establecimiento, Dirección y Teléfono
Hasta este punto, hemos estado utilizando diferentes estrategias para limpiar cada variable por separado. Sin embargo, todavía debemos intentar identificar posibles errores de digitación utilizando diferentes variables en conjunto.

Primero, vamos a identificar pequeños errores de digitación para los nombres de establecimiento. Estaremos utilizando fuzzy matching con un treshold de 0.95. Este fue decidido luego de algunas pruebas y errores en un inciso anterior, dónde algunos institutos diferentes aparecían dentro de la lista al tener una similitud en ej. "Primaria" vs "Preprimaria". Vamos a utilizar matching del 0.95, luego para terminar de identificar el establecimiento estaremos utilizando número telefónico y director. D

In [53]:
from difflib import get_close_matches
from collections import defaultdict
import pandas as pd

def corregir_nombres_similares(datasets, threshold=0.95):
    """
    Corrige nombres de establecimientos similares (pero no idénticos) para cada grupo con el mismo teléfono y director.
    """

    # Paso 1: Agrupar por teléfono + director
    grupos = defaultdict(list)
    for nombre_df, df in datasets.items():
        if all(col in df.columns for col in ['ESTABLECIMIENTO', 'TELEFONO', 'DIRECTOR']):
            for idx, row in df.iterrows():
                if pd.notna(row['TELEFONO']) and pd.notna(row['DIRECTOR']):
                    clave = (str(row['TELEFONO']), str(row['DIRECTOR']).upper())
                    nombre = str(row['ESTABLECIMIENTO']).upper().strip()
                    grupos[clave].append((nombre_df, idx, nombre))

    # Paso 2: Crear un mapeo de corrección
    reemplazos = {}

    for clave, lista in grupos.items():
        nombres_unicos = list(set([nombre for _, _, nombre in lista]))
        clusters = []

        for nombre in nombres_unicos:
            # Intentar meter el nombre en un cluster existente
            found_cluster = False
            for cluster in clusters:
                if get_close_matches(nombre, cluster, n=1, cutoff=threshold):
                    cluster.append(nombre)
                    found_cluster = True
                    break
            if not found_cluster:
                clusters.append([nombre])

        # Para cada cluster, seleccionar el nombre canónico y mapear los demás
        for cluster in clusters:
            if len(cluster) < 2:
                continue  # No hay errores si sólo hay uno
            nombre_referencia = max(cluster, key=len)  # El más largo como referencia
            for variante in cluster:
                if variante != nombre_referencia:
                    reemplazos[variante] = nombre_referencia

    # Paso 3: Reemplazar en los datasets
    for df in datasets.values():
        if 'ESTABLECIMIENTO' in df.columns:
            df['ESTABLECIMIENTO'] = df['ESTABLECIMIENTO'].apply(
                lambda x: reemplazos.get(str(x).upper().strip(), x)
            )

    # Paso 4: Reporte de cambios
    cambios = [{'VARIANTE': k, 'CORREGIDO A': v} for k, v in reemplazos.items()]
    return pd.DataFrame(cambios).drop_duplicates()

correcciones = corregir_nombres_similares(datasets, threshold=0.95)

if not correcciones.empty:
    print("✅ Correcciones aplicadas:")
    display(correcciones)
else:
    print("✅ No se encontraron nombres similares para corregir.")


✅ Correcciones aplicadas:


,VARIANTE,CORREGIDO A
0,CENTRO DE ESTUDIOS PREUNIVERSITARIO CAHABON CE...,CENTRO DE ESTUDIOS PREUNIVERSITARIOS CAHABON C...
1,COLEGIO INTERCIENTIFICO MULTILINGUE Y MULTIVER...,COLEGIO INTERCIENTIFICOMULTILINGUE Y MULTIDIVE...
2,COLEGIO PRIVADO MIXTO IMPERIAL,COLEGIO PRIVADO MIXTO IMPERIAL2
3,COLEGIO EVANGELICO TECNOLOGICO PRIVADO MIXTO B...,COLEGIO EVANGELICO TECNOLOGICO PRIVADO MIXTO B...
4,COLEGIO CRISTIANO PREUNIVERSITARIO EL REFORMADOR,COLEGIO CRISTIANO PREUNIVERSITARIO EL REFORMADOS
5,INSTITUTO PRIVADO RAFAEL AREVALO MARTINEZ,INSTITUTO PRIVADO RAFAEN AREVALO MARTINEZ
6,COLEGIO PARTICULAR MIXTO CENTRO INTEGRAL SISTE...,COLEGIO PARTICULAR MIXTO CENTRO INTREGRAL SIST...
7,INSTITUTO MUNICIPAL DE EDUCACION BASICA Y DIVE...,INSTITUTO MUNICIPAL DE EDUCACION BASICA Y DIVE...
8,INSTITUTO TECNOLGIA Y CIENCIA,INSTITUTO TECNOLOGIA Y CIENCIA
9,COLEGIO BROWNS,COLEGIO BROWN S


Luego de un review manual, llegamos a la conclusión que todos los cambios realizados son verídicos y fueron realizados correctamente. Ahora, podemos revisar que las direcciones sean las mismas para establecimientos que tengan nombres idénticos.

In [54]:
from difflib import get_close_matches
from collections import defaultdict
import pandas as pd

def corregir_direcciones_similares(datasets, threshold=0.95):
    """
    Corrige direcciones similares (pero no idénticas) dentro del mismo establecimiento,
    siempre que la similitud sea mayor al threshold.
    """

    # Paso 1: Agrupar por nombre de establecimiento
    grupos = defaultdict(list)
    for nombre_df, df in datasets.items():
        if all(col in df.columns for col in ['ESTABLECIMIENTO', 'DIRECCION']):
            for idx, row in df.iterrows():
                if pd.notna(row['ESTABLECIMIENTO']) and pd.notna(row['DIRECCION']):
                    clave = str(row['ESTABLECIMIENTO']).upper().strip()
                    direccion = str(row['DIRECCION']).upper().strip()
                    grupos[clave].append((nombre_df, idx, direccion))

    reemplazos = {}

    for establecimiento, lista in grupos.items():
        direcciones_unicas = list(set([direccion for _, _, direccion in lista]))
        if len(direcciones_unicas) <= 1:
            continue  # Nada que comparar

        clusters = []

        for direccion in direcciones_unicas:
            # Solo comparar contra otras que no son iguales
            found_cluster = False
            for cluster in clusters:
                for existente in cluster:
                    if direccion != existente and get_close_matches(direccion, [existente], n=1, cutoff=threshold):
                        cluster.append(direccion)
                        found_cluster = True
                        break
                if found_cluster:
                    break
            if not found_cluster:
                clusters.append([direccion])

        for cluster in clusters:
            if len(cluster) < 2:
                continue
            ref = max(cluster, key=len)  # o más frecuente si prefieres
            for variante in cluster:
                if variante != ref:
                    reemplazos[variante] = ref

    # Paso 3: Reemplazar en datasets
    for df in datasets.values():
        if 'DIRECCION' in df.columns:
            df['DIRECCION'] = df['DIRECCION'].apply(
                lambda x: reemplazos.get(str(x).upper().strip(), x)
            )

    # Paso 4: Reportar cambios
    cambios = [{'VARIANTE': k, 'CORREGIDO A': v} for k, v in reemplazos.items()]
    return pd.DataFrame(cambios).drop_duplicates()

correcciones_direcciones = corregir_direcciones_similares(datasets, threshold=0.95)

if not correcciones_direcciones.empty:
    print("✅ Direcciones corregidas por similitud (excluyendo las ya idénticas):")
    display(correcciones_direcciones)
else:
    print("✅ No se encontraron direcciones similares no idénticas para corregir.")

✅ Direcciones corregidas por similitud (excluyendo las ya idénticas):


,VARIANTE,CORREGIDO A
0,9 CALLEJON B 12-94 ZONA 4 SECTOR LOS OLIVOS EL...,9O CALLEJON B 12-94 ZONA 4 SECTOR LOS OLIVOS E...
1,CAERIO SANTA RITA PACHIPA,CASERIO SANTA RITA PACHIPA
2,2 AVENIDA 11 CALLE ZONA1,2 AVENIDA 11 CALLE ZONA 1
3,CARRETERA INTERAMERICANA P6 ZONA 4,CARRETERA INTERAMERICACANA P6 ZONA 4
4,3 CALLE1-22 ZONA 3,3 CALLE 1-22 ZONA 3
...,...,...
115,CANTON CONCEPCION SECTOR LA MANCHURIA,CANTON COCCEPCION SECTOR LA MANCHURIA
116,CANTON LA PAZONA RANCHO LOS OLIVOS CARETERA A ...,CANTON LA PAZONA RANCHO LOS OLIVOS CARRETERA A...
117,7 CALLE 8 AVENIDA 7-49 ZONA 3 BARRIO EL CHAPARON,7 CALLE 8 AVENIDA 7-49 ZONA 3 BARRIO EL CHAPARRON
118,3 CALLE 5-97 ZONA 1,3- CALLE 5-97 ZONA 1


Luego de una revisión manual, las direcciones actualizadas contenían errores de digitación y representan una misma dirección. Gracias a estos pasos adicionales, evitamos establecimientos y direcciones con errores de digitación y tenemos una mejor consistencia en nuestros datos.

## Merging de Establecimientos y "Checkpoint"
Los establecimientos tendrán múltiples entradas en el dataset final, esto ya que cada combinación de "Status" "Modalidad" "Jornada" y "Plan" se refiere a su propio y único plan de estudios. Si fuéramos a utilizar un Merge, luego quedaría un dataset donde se asume que el establecimiento ofrece planes de cualquier combinación presente. Por ejemplo, tendríamos un establecimiento con un plan Diario Matutino, y Fin De Semana Vespertino. Luego de encodear las variables, tendríamos datos que nos indican la existencia de un plan Fin de Semana Matutino.

Recapitulando hasta el momento, hemos limpiado las variables hasta Director enfocados en la consistencia de los datos. El dataset actualmente se ve de esta manera:

In [55]:
first_key = next(iter(datasets))
first_df = datasets[first_key]
print(f"📄 First dataset: {first_key}")
display(first_df.head(10))


📄 First dataset: datos_chiquimula


,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
0,20-01-0024-46,20-001,CHIQUIMULA,CHIQUIMULA,ESCUELA DE CIENCIAS COMERCIALES NOCTURNA,10 AVENIDA 3-71 ZONA 1,79422150,CESAR ADALBERTO NOGUERA JACOME,HECTOR ALIDIO CERON BRENES,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,NOCTURNA,DIARIO(REGULAR),CHIQUIMULA
1,20-01-0029-46,20-001,CHIQUIMULA,CHIQUIMULA,INSTITUTO DIVERSIFICADO ADS AL INEB DR DAVID G...,2 CALLE ENTRE 11 12 AVENIDAS ZONA 1,79420395,CESAR ADALBERTO NOGUERA JACOME,ROMEO RIVERA CHACON,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),CHIQUIMULA
2,20-01-0030-46,20-001,CHIQUIMULA,CHIQUIMULA,ESCUELA NACIONAL DE MAESTROS DE EDUCACION MUSI...,2 CALLE 12 AVENIDA ZONA 1,41942927,CESAR ADALBERTO NOGUERA JACOME,JOSE VICENTE REYES SOLIS,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,NOCTURNA,DIARIO(REGULAR),CHIQUIMULA
3,20-01-0031-46,20-024,CHIQUIMULA,CHIQUIMULA,ESCUELA SECUNDARIA PRIVADA MIXTA DE ORIENTE ESDEO,2 CALLE 4-40 ZONA 1,79420290,SILVIA MARILENA BUEZO MARTINEZ,ERWIN LEONEL COLINDRES MONROY,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),CHIQUIMULA
4,20-01-0032-46,20-024,CHIQUIMULA,CHIQUIMULA,COLEGIO EVANGELICO MIXTO AMIGOS,10 AVENIDA 5-38 ZONA 1,79420116,SILVIA MARILENA BUEZO MARTINEZ,ROXANA LISBETH URRUTIA PINTO,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),CHIQUIMULA
5,20-01-0033-46,20-027,CHIQUIMULA,CHIQUIMULA,INSTITUTO TECNOLOGICO PRIVADO DE ORIENTE,4 AVENIDA 4-20 ZONA 1,79425959,SILVIA MARILENA BUEZO MARTINEZ,SALOMON ELIASIB ALVAREZ CORDON,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,DOBLE,DIARIO(REGULAR),CHIQUIMULA
6,20-01-0034-46,20-001,CHIQUIMULA,CHIQUIMULA,INSTITUTO NORMAL PARA SENORITAS DE ORIENTE,4 CALLE 6-00 ZONA 1,59799216,CESAR ADALBERTO NOGUERA JACOME,MIRNA ELIZABETH QUIROA LOPEZ,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),CHIQUIMULA
7,20-01-0036-46,20-001,CHIQUIMULA,CHIQUIMULA,INSTITUTO NORMAL PARA VARONES DE ORIENTE,6 AVE 4-10 ZONA 1,79420425,CESAR ADALBERTO NOGUERA JACOME,BALVINO CHACON PEREZ,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),CHIQUIMULA
8,20-01-0038-46,20-027,CHIQUIMULA,CHIQUIMULA,LICEO LA SALLE,6 CALLE 3-61 ZONA 1,79420328,SILVIA MARILENA BUEZO MARTINEZ,JUAN MODESTO SICAY TOC,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),CHIQUIMULA
9,20-01-0040-46,20-024,CHIQUIMULA,CHIQUIMULA,COLEGIO PRIVADO MIXTO SAGRADA FAMILIA,6 AVE 5-68 ZONA 1,79420232,SILVIA MARILENA BUEZO MARTINEZ,DAMARYS JUDITH DIAZ TEJADA,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),CHIQUIMULA


De momento, queda pendiente la limpieza de las variables categóricas restantes y eliminación de duplicados. La eliminación de duplicados la estamos manteniendo hasta el final, ya que primero debemos asegurarnos que un caso como "Doble" y "Dóble" no genere datos duplicados debido a errores de ingreso.

## Variables categóricas

### Limpieza y estandarización de categóricas
Forzamos mayúsculas, desacentuamos, quitamos basura (paréntesis, dobles espacios).

Mapeamos sinónimos/typos a los únicos valores válidos.

Todo lo que no calce → NaN (excepto AREA, donde “sin especificar” se mapea explícitamente).

In [56]:
import pandas as pd, re, unicodedata
from pathlib import Path

# --- utilidades de texto ---
def _deaccent(s: str) -> str:
    return unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")

def norm_cat(x):
    if pd.isna(x): return None
    s = str(x).strip().upper()
    s = _deaccent(s)
    s = re.sub(r'\s*\([^)]*\)', '', s)     
    s = re.sub(r'\s+', ' ', s)             # colapsa espacios
    s = s.replace('”', '"').replace('“', '"')
    return s

# --- valores válidos que pediste ---
VALID_SECTOR   = {"OFICIAL", "MUNICIPAL", "PRIVADO", "COOPERATIVA"}
VALID_AREA     = {"URBANA", "RURAL", "SIN ESPECIFICAR"}
VALID_MODAL    = {"MONOLINGUE", "BILINGUE"}
VALID_JORNADA  = {"MATUTINA", "VESPERTINA", "NOCTURNA", "DOBLE", "SIN JORNADA", "INTERMEDIA"}
VALID_PLAN     = {"DIARIO", "A DISTANCIA", "SEMIPRESENCIAL", "FIN DE SEMANA"}

# --- diccionarios de normalización (sinónimos/typos comunes) ---
MAP_SECTOR = {
    "OFICIA": "OFICIAL", "OFIC": "OFICIAL",
    "MUNI": "MUNICIPAL",
    "PRIV": "PRIVADO", "PARTICULAR": "PRIVADO",
    "COOP": "COOPERATIVA"
}

MAP_AREA = {
    "URBAN": "URBANA", "URB": "URBANA",
    "RURAL": "RURAL",
    "SIN ESPECIF": "SIN ESPECIFICAR", "NO ESPECIFICADO": "SIN ESPECIFICAR",
    "S/E": "SIN ESPECIFICAR", "SIN": "SIN ESPECIFICAR"
}

MAP_MODAL = {
    "BILINGUEE": "BILINGUE", "BIL": "BILINGUE",
    "MONO": "MONOLINGUE", "MONOLINGUEE": "MONOLINGUE"
}

MAP_JORNADA = {
    "DOBLE JORNADA": "DOBLE",
    "MAT": "MATUTINA",
    "VESP": "VESPERTINA", "VESPERTINO": "VESPERTINA",
    "NOCT": "NOCTURNA",
    "INTERMEDIO": "INTERMEDIA",
    "SIN": "SIN JORNADA", "NINGUNA": "SIN JORNADA"
}

MAP_PLAN = {
    "DIARIO REGULAR": "DIARIO", "REGULAR": "DIARIO", "DIARIO( REGULAR )": "DIARIO",
    "DISTANCIA": "A DISTANCIA", "A-DISTANCIA": "A DISTANCIA",
    "SEMI PRESENCIAL": "SEMIPRESENCIAL", "SEMI-PRESENCIAL": "SEMIPRESENCIAL", "SEMIPRES": "SEMIPRESENCIAL",
    "FINDESEMANA": "FIN DE SEMANA", "FDS": "FIN DE SEMANA"
}

def _map_value(x, valid_set, mapping_extra):
    if x is None: return None
    # mapeos puntuales
    if x in mapping_extra: 
        x = mapping_extra[x]
    # si coincide exacto con válidos → ok
    if x in valid_set:
        return x
    # intentos suaves: recortar/normalizar espacios
    y = re.sub(r'\s+', ' ', x).strip()
    if y in mapping_extra: y = mapping_extra[y]
    if y in valid_set: return y
    return None  # fuera de catálogo

def clean_categoricals(df):
    if df is None or not isinstance(df, pd.DataFrame):
        # defensa: nunca retornes None
        return pd.DataFrame()

    df = df.copy()
    for col in ["SECTOR", "AREA", "MODALIDAD", "JORNADA", "PLAN"]:
        if col in df.columns:
            df[col] = df[col].map(norm_cat)  # deja en mayúsculas sin tildes

    # mapeos exactos
    if "SECTOR" in df:
        df["SECTOR"] = df["SECTOR"].map(lambda x: _map_value(x, VALID_SECTOR, MAP_SECTOR))
    if "AREA" in df:
        df["AREA"] = df["AREA"].map(lambda x: _map_value(x, VALID_AREA, MAP_AREA))
    if "MODALIDAD" in df:
        df["MODALIDAD"] = df["MODALIDAD"].map(lambda x: _map_value(x, VALID_MODAL, MAP_MODAL))
    if "JORNADA" in df:
        df["JORNADA"] = df["JORNADA"].map(lambda x: _map_value(x, VALID_JORNADA, MAP_JORNADA))

    # --- PLAN: primero intentamos exacto, luego reglas “contains” ---
    if "PLAN" in df:
        df["PLAN"] = df["PLAN"].map(lambda x: _map_value(x, VALID_PLAN, MAP_PLAN))

        def plan_contains_fix(x):
            if x in VALID_PLAN or x is None:
                return x
            s = str(x)
            if "DISTANCIA" in s:
                return "A DISTANCIA"
            if "SEMANA" in s:          # FIN DE SEMANA
                return "FIN DE SEMANA"
            if "SEMI" in s:            # SEMIPRESENCIAL / SEMI PRESENCIAL
                return "SEMIPRESENCIAL"
            if "DIARIO" in s or "REGULAR" in s:
                return "DIARIO"
            return None

        df["PLAN"] = df["PLAN"].map(plan_contains_fix)

    # reporte
    def _report(col, valid):
        if col in df:
            bad = df[col].isna().sum()
            print(f"{col}: {bad} fuera de catálogo" + (f" (válidos: {sorted(list(valid))})" if bad else ""))
    _report("SECTOR", VALID_SECTOR)
    _report("AREA", VALID_AREA)
    _report("MODALIDAD", VALID_MODAL)
    _report("JORNADA", VALID_JORNADA)
    _report("PLAN", VALID_PLAN)

    return df


### Derivadas + dummies (con tipos específicos)
sector_* → 0/1 (int).

area_*, modalidad_*, jornada_*, plan_* → boolean (True/False).

es_publico, es_rural, es_urbano, tiene_telefono → 0/1 (int).

tipo_establecimiento (heurística) y longitud_nombre (entero).

In [57]:
# --- teléfono válido: 8 dígitos ---
def is_valid_phone(x):
    if pd.isna(x): return False
    s = re.sub(r'\D', '', str(x))
    return len(s) == 8

def categorize_establishment(name):
    u = str(name).upper()
    if any(w in u for w in ["INSTITUTO", "COLEGIO"]):
        return "INSTITUTO_COLEGIO"
    elif any(w in u for w in ["ESCUELA", "LICEO"]):
        return "ESCUELA_LICEO"
    elif any(w in u for w in ["CENTRO", "NUCLEO", "C.E.B.", "CEB "]):
        return "CENTRO"
    else:
        return "OTRO"

def add_features_and_dummies(df):
    df = df.copy()

    # --- derivadas numéricas/binarias ---
    df["es_publico"]   = df["SECTOR"].isin(["OFICIAL", "MUNICIPAL"]).astype(int)
    df["es_rural"]     = (df["AREA"] == "RURAL").astype(int)
    df["es_urbano"]    = (df["AREA"] == "URBANA").astype(int)
    df["tiene_telefono"]= df["TELEFONO"].map(is_valid_phone).astype(int)

    df["tipo_establecimiento"] = df["ESTABLECIMIENTO"].map(categorize_establishment)
    df["longitud_nombre"]      = df["ESTABLECIMIENTO"].astype(str).str.len().astype("Int64")

    # --- dummies con tipos solicitados ---
    # sector_* -> 0/1
    sector_dum = pd.get_dummies(df["SECTOR"], prefix="sector", dtype="uint8")
    for col in ["sector_COOPERATIVA","sector_MUNICIPAL","sector_OFICIAL","sector_PRIVADO"]:
        if col not in sector_dum.columns:
            sector_dum[col] = 0  # garantizar columnas aunque no existan en el df
    sector_dum = sector_dum[["sector_COOPERATIVA","sector_MUNICIPAL","sector_OFICIAL","sector_PRIVADO"]]

    # area_* (boolean)
    area_dum = pd.get_dummies(df["AREA"], prefix="area").astype("boolean")
    for col in ["area_RURAL","area_URBANA"]:
        if col not in area_dum.columns:
            area_dum[col] = False
    area_dum = area_dum[["area_RURAL","area_URBANA"]]

    # modalidad_* (boolean)
    mod_dum = pd.get_dummies(df["MODALIDAD"], prefix="modalidad").astype("boolean")
    for col in ["modalidad_BILINGUE","modalidad_MONOLINGUE"]:
        if col not in mod_dum.columns:
            colname = col
            mod_dum[colname] = False
    mod_dum = mod_dum[["modalidad_BILINGUE","modalidad_MONOLINGUE"]]

    # jornada_* (boolean)
    jornadas = ["DOBLE","INTERMEDIA","MATUTINA","NOCTURNA","SIN JORNADA","VESPERTINA"]
    jor_dum = pd.get_dummies(df["JORNADA"], prefix="jornada").astype("boolean")
    # garantiza todas las columnas en orden
    cols_j = [f"jornada_{j}" for j in jornadas]
    for c in cols_j:
        if c not in jor_dum.columns:
            jor_dum[c] = False
    jor_dum = jor_dum[cols_j]

    # plan_* (boolean)
    planes = ["A DISTANCIA", "DIARIO", "FIN DE SEMANA", "SEMIPRESENCIAL"]
    plan_dum = pd.get_dummies(df["PLAN"], prefix="plan").astype("boolean")
    cols_p = [f"plan_{p}" for p in planes]
    # mapea espacios del sufijo como están en los valores
    cols_p = [c.replace(" ", " ") for c in cols_p]
    for c in cols_p:
        if c not in plan_dum.columns:
            plan_dum[c] = False
    plan_dum = plan_dum[cols_p]

    # --- concatenar al df limpio ---
    df_out = pd.concat([df, sector_dum, area_dum, mod_dum, jor_dum, plan_dum], axis=1)

    return df_out, sector_dum, area_dum, mod_dum, jor_dum, plan_dum


### Pipeline sobre todos los CSV y dataset aparte de dummies

mineduc_combined_clean.csv (categóricas estandarizadas + derivadas).

mineduc_dummies.csv (solo dummies + identificador CODIGO para merge fácil).

In [58]:
def process_all_and_save(
    input_dir="data/csv",
    out_dir="data/cleaned",
    make_dummies_dataset=True
):
    in_path = Path(input_dir)
    out_path = Path(out_dir)
    out_path.mkdir(parents=True, exist_ok=True)

    csvs = list(in_path.glob("*.csv"))
    print(f"Procesando {len(csvs)} archivos...")

    cleaned_list = []
    dummies_list = []

    for f in csvs:
        df = pd.read_csv(f, dtype=str)  # leer como texto para no romper códigos/teléfono
        obj_cols = df.select_dtypes(include="object").columns
        df[obj_cols] = df[obj_cols].apply(lambda s: s.str.strip())

        # limpia categóricas exactamente a catálogo
        df_clean = clean_categoricals(df)

        if df_clean is None or df_clean.empty:
            print(f"⚠️ {f.name}: dataset vacío tras limpieza; se omite.")
            continue

        # agrega derivadas y dummies
        df_full, sector_dum, area_dum, mod_dum, jor_dum, plan_dum = add_features_and_dummies(df_clean)

        # guarda versión limpia individual si quieres
        (out_path / f"{f.stem}_clean.csv").write_text(
            df_full.to_csv(index=False, encoding="utf-8"), encoding="utf-8"
        )

        cleaned_list.append(df_full)

        if make_dummies_dataset:
            # dataset de dummies con clave para merge: CODIGO (si existe) o índice
            key = "CODIGO" if "CODIGO" in df_full.columns else None
            dummies = pd.concat([sector_dum, area_dum, mod_dum, jor_dum, plan_dum], axis=1)
            if key:
                dummies.insert(0, "CODIGO", df_full[key].astype(str))
            else:
                dummies.insert(0, "row_id", range(len(dummies)))
            dummies_list.append(dummies)

    # combinados
    combined = pd.concat(cleaned_list, ignore_index=True)
    combined_path = out_path / "mineduc_combined_clean.csv"
    combined.to_csv(combined_path, index=False, encoding="utf-8")
    print(f"✅ Limpio combinado: {combined.shape} → {combined_path}")

    if make_dummies_dataset and dummies_list:
        dummies_combined = pd.concat(dummies_list, ignore_index=True)
        # tipa los booleanos correctamente
        bool_cols = [c for c in dummies_combined.columns if c.startswith(("area_", "modalidad_", "jornada_", "plan_"))]
        dummies_combined[bool_cols] = dummies_combined[bool_cols].astype("boolean")

        dum_path = out_path / "mineduc_dummies.csv"
        dummies_combined.to_csv(dum_path, index=False, encoding="utf-8")
        print(f"✅ Dummies combinado: {dummies_combined.shape} → {dum_path}")

    return combined


Se ejecuta:

In [59]:
combined_clean = process_all_and_save("data/csv", "data/cleaned", make_dummies_dataset=True)

Procesando 23 archivos...
SECTOR: 0 fuera de catálogo
AREA: 0 fuera de catálogo
MODALIDAD: 0 fuera de catálogo
JORNADA: 0 fuera de catálogo
PLAN: 0 fuera de catálogo
SECTOR: 0 fuera de catálogo
AREA: 0 fuera de catálogo
MODALIDAD: 0 fuera de catálogo
JORNADA: 0 fuera de catálogo
PLAN: 2 fuera de catálogo (válidos: ['A DISTANCIA', 'DIARIO', 'FIN DE SEMANA', 'SEMIPRESENCIAL'])
SECTOR: 0 fuera de catálogo
AREA: 0 fuera de catálogo
MODALIDAD: 0 fuera de catálogo
JORNADA: 0 fuera de catálogo
PLAN: 2 fuera de catálogo (válidos: ['A DISTANCIA', 'DIARIO', 'FIN DE SEMANA', 'SEMIPRESENCIAL'])
SECTOR: 0 fuera de catálogo
AREA: 0 fuera de catálogo
MODALIDAD: 0 fuera de catálogo
JORNADA: 0 fuera de catálogo
PLAN: 4 fuera de catálogo (válidos: ['A DISTANCIA', 'DIARIO', 'FIN DE SEMANA', 'SEMIPRESENCIAL'])
SECTOR: 0 fuera de catálogo
AREA: 0 fuera de catálogo
MODALIDAD: 0 fuera de catálogo
JORNADA: 0 fuera de catálogo
PLAN: 0 fuera de catálogo
SECTOR: 0 fuera de catálogo
AREA: 0 fuera de catálogo
MODA

### Eliminar duplicados

In [60]:
duplicate_check_cols = [col for col in combined_clean.columns if col != 'CODIGO']

print("Análisis de duplicados:")
print(f"Filas iniciales: {len(combined_clean):,}")

duplicated_mask = combined_clean.duplicated(subset=duplicate_check_cols, keep=False)
duplicate_count = duplicated_mask.sum()

if duplicate_count > 0:
    print(f"Duplicados encontrados: {duplicate_count:,} filas")
    
    duplicate_rows = combined_clean[duplicated_mask]
    duplicate_groups = duplicate_rows.groupby(duplicate_check_cols)
    
    print(f"Grupos de duplicados: {len(duplicate_groups):,}")
    print("\nEjemplos de duplicados:")
    
    sample_count = 0
    for group_key, group_df in duplicate_groups:
        if sample_count >= 3:
            break
        print(f"\n   Grupo {sample_count + 1}: {len(group_df)} duplicados")
        print(f"   Establecimiento: {group_df['ESTABLECIMIENTO'].iloc[0]}")
        print(f"   Municipio: {group_df['MUNICIPIO'].iloc[0]}")
        print(f"   CODIGOs: {', '.join(group_df['CODIGO'].astype(str))}")
        sample_count += 1
    
    if len(duplicate_groups) > 3:
        print(f"   ... y {len(duplicate_groups) - 3} grupos más")
    
    combined_clean_no_dup = combined_clean.drop_duplicates(subset=duplicate_check_cols, keep='first')
    
    final_count = len(combined_clean_no_dup)
    removed_count = len(combined_clean) - final_count
    
    print(f"\nDuplicados eliminados:")
    print(f"Filas finales: {final_count:,}")
    print(f"Filas eliminadas: {removed_count:,}")
    print(f"Reducción: {(removed_count/len(combined_clean))*100:.2f}%")

    combined_clean = combined_clean_no_dup.copy()
    
else:
    print("No se encontraron duplicados")

output_path = Path("data/cleaned")
output_path.mkdir(parents=True, exist_ok=True)

final_file = output_path / "mineduc_final_clean.csv"
combined_clean.to_csv(final_file, index=False, encoding='utf-8')

print(f"\nDataset final guardado en: {final_file}")
print(f"Forma final del dataset: {combined_clean.shape}")

Análisis de duplicados:
Filas iniciales: 6,590
Duplicados encontrados: 272 filas
Grupos de duplicados: 134

Ejemplos de duplicados:

   Grupo 1: 2 duplicados
   Establecimiento: LICEO INTEGRAL PALENCIANO
   Municipio: PALENCIA
   CODIGOs: 01-05-8549-46, 01-05-8550-46

   Grupo 2: 2 duplicados
   Establecimiento: COLEGIO BELLO AMANECER 2
   Municipio: ZONA 18
   CODIGOs: 00-18-0211-46, 00-18-0257-46

   Grupo 3: 2 duplicados
   Establecimiento: LICEO CUMORAH PINOS
   Municipio: ZONA 18
   CODIGOs: 00-18-0154-46, 00-18-0156-46
   ... y 131 grupos más

Duplicados eliminados:
Filas finales: 6,452
Filas eliminadas: 138
Reducción: 2.09%

Dataset final guardado en: data/cleaned/mineduc_final_clean.csv
Forma final del dataset: (6452, 41)
